In [1]:
import requests
from bs4 import BeautifulSoup
import datetime
import os
import pymysql
from tqdm import tqdm
import numpy as np
import time

In [2]:
ip_address = os.environ['AWS_IP']
PASSWORD = os.environ['SQL_PWD_AWS']

con = pymysql.connect(
    user='edward', 
    passwd=PASSWORD,
    host=ip_address, 
    db='community', 
    charset='utf8'
)

In [3]:
BASE_URL = 'https://www.fmkorea.com'

In [4]:
def get_url_date(li):
#     url = BASE_URL + li.find('h3').find('a')['href']
    url = li.find('h3').find('a')['href']
    date = li.find('span', {'class': 'regdate'}).text.strip()
    date = date.replace('.', '-')
    return [url, date]

In [5]:
def get_urls(i):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    params = {'mid': 'best', 'page': i}
    result = requests.get(BASE_URL + '/index.php', params = params, headers = headers)
    soup = BeautifulSoup(result.text, 'lxml')
    table = soup.find('div', {'class': 'fm_best_widget _bd_pc'})
    lists = table.find_all('li')
    return [get_url_date(li) for li in lists]

In [6]:
def commit_sql(data):
    with  con.cursor() as cursor:
        cursor.executemany("insert into urls_fmk(url, date) values (%s, %s)", data )
        con.commit()

In [14]:
start = 8580
end = 140

In [16]:
for i in tqdm(range(start, end, -1)):
    data = get_urls(i)
    commit_sql(data)
    time.sleep(np.random.random()*5)

  1%|          | 45/8440 [02:08<6:29:23,  2.78s/it]

AttributeError: 'NoneType' object has no attribute 'find_all'